<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Quantum_Machine_Learning_(QML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane

In [ ]:
import pennylane as qml
import torch
import torch.nn as nn

# Initialize a quantum device with 2 qubits
dev = qml.device("default.qubit", wires=2)

# Define the quantum circuit using a QNode
@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(2))
    qml.templates.BasicEntanglerLayers(weights, wires=range(2))
    return qml.expval(qml.PauliZ(0))

# Define the quantum neural network class
class QuantumNet(nn.Module):
    def __init__(self, n_qubits, n_layers):
        super(QuantumNet, self).__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.weight_shapes = {"weights": (n_layers, n_qubits)}
        self.quantum_circuit = qml.qnn.TorchLayer(quantum_circuit, self.weight_shapes)

    def forward(self, x):
        return self.quantum_circuit(x)

# Initialize the QuantumNet model with 2 qubits and 3 layers
model = QuantumNet(n_qubits=2, n_layers=3)